In [1]:
# Temp for local development
import ssl
import os
# os.environ['PYSPARK_PYTHON'] = '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6'
import tinydb as tinydb
ssl._create_default_https_context = ssl._create_unverified_context

In [7]:
%%bash
# The Search for Categorical Correlation
# https://towardsdatascience.com/the-search-for-categorical-correlation-a1cf7f1888c9
# https://github.com/shakedzy/dython
# http://shakedzy.xyz/dython/
if ! pip3 list | grep dython; then
    pip3 install dython
fi

dython              0.6.1


In [8]:
%%bash
if ! pip3 list | grep tinydb; then
    pip3 install tinydb
fi

tinydb              4.2.0


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn

from pyspark import SparkContext, SparkFiles, SQLContext
from pyspark.sql.types import StructType, StructField, StringType, BooleanType

from pyspark.sql import functions as F
from pyspark.sql import types as T

from itertools import chain
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
# https://github.com/shakedzy/dython
from dython.nominal import associations

In [ ]:
sc = SparkContext.getOrCreate()
spark = SQLContext(sc)

In [ ]:
import os
import json
dataDir = '/Users/duane.hinkley/PycharmProjects/c772-capstone-project/jupyter/.data/'

if not os.path.exists(dataDir):
    os.makedirs(dataDir)

def init_df_raw():
    dfRaw = import_by_url('https://github.com/dlhinkley/c772-capstone-project/raw/master/data/assessment_items.csv')

    # Save to reuse
    save_df(dfRaw, 'dfRaw')

    # Only keep "fully scored" items
    # Filter to learner_attempt_status = 'fully scored'
    dfFlt = dfRaw.filter(F.col('learner_attempt_status') == 'fully scored')

    # Change Date Fields from String to Timestamp Type
    types = get_var_types()
    for f in types['intervalVars']:
      dfFlt = dfFlt.withColumn(f, F.col(f).cast(T.TimestampType() ) )

    # Set default date values to null (years 2999 and 1900)
    # Set empty dates to null
    for f in types['intervalVars']:
      # Change to empty if date is more than 30 months in past or future
      dfFlt = dfFlt.withColumn(f, F.when( F.abs(F.months_between(F.col(f), F.current_timestamp() )) > 30, None ).otherwise( F.col(f) ) )

    # Save to reuse
    save_df(dfFlt, 'dfFlt')

    return dfFlt


def init_df_desc():
    dfDesc = import_by_url('https://github.com/dlhinkley/c772-capstone-project/raw/master/data/descriptions.csv')
    save_df(dfDesc, 'dfDesc')
    return dfDesc


def save_df(df, name):
    df.repartition(1).write.mode('overwrite').parquet(dataDir + name + ".parquet")

def save_dict(data, name):
    with open(dataDir + name + ".json", "w") as f:
      json.dump(data, f)

def load_dict(name):
    with open(dataDir + name + ".json") as f:
        out = json.load(f)

    return out

def load_df(name):
    return spark.read.parquet(dataDir + name + ".parquet")

In [ ]:
def get_non_string_vars():
    global shared
    return group.identifierVars + group.continousVars + group.intervalVars + group.binaryVars

def get_all_vars():
        return group.nominalVars + get_non_string_vars();

In [ ]:
from tinydb import TinyDB, Query
# Create Todo list
def init_todo():
  global td, dataDir
  td = TinyDB(dataDir + 'todo.json')

def add_todo(desc):
  global td
  q = Query()
  if not td.contains(q.todo == desc):
      td.insert({'todo': desc, 'finished': False})

def list_todo(finished = None):
  global td

  for item in td:
    if (finished != None):
        if (item['finished'] == finished):
           print(item)
    else:
        print(item)

def finish_todo(desc):
  global td
  q = Query()
  td.update({'finished': True}, q.todo == desc)

def delete_todo(desc):
  global td
  q = Query()
  td.remove(q.todo == desc)


init_todo()

In [ ]:
import os
from pyspark import SparkFiles
from datetime import datetime

def import_by_url(url):
  # Given a url to a csv file, import and return a dataframe
  #
  sc.addFile(url)
  filename = os.path.basename(url)
  file = "file://" + SparkFiles.get(filename)
  return spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(file)


def filter_default(dfIn, f1, f2):
  # Given a dataframe and two date field names, returns the dataframe removing records
  # where the f1 or f2 columns equal a default date
  defaultDates = ["2999-01-01 00:00:00", "1900-01-01 00:00:00"]
  return dfIn.filter( ~F.col(f1).isin(defaultDates) & ~F.col(f2).isin(defaultDates) )


def date_stats(dfIn, f1, f2):
  # Given a dataframe and two date field names, returns a new dataframe with the difference between
  # the dates in minutes, hours and minutes
  dfOut = filter_default(dfIn, f1, f2)

  dfOut = dfOut.withColumn("minues", (F.col(f1).cast("long") - F.col(f2).cast("long"))/60.).select(f1, f2, "minues")

  dfOut = dfOut.withColumn("hours", (F.col(f1).cast("long") - F.col(f2).cast("long"))/3600.).select(f1, f2, "hours", "minues")

  return dfOut.withColumn("days", (F.col(f1).cast("long") - F.col(f2).cast("long"))/86400.).select("days", "hours", "minues")


def annotate_plot(ax):
  # Add total labels to plot
  for p in ax.patches:
      ax.annotate(
        round(p.get_height(), 2),
        (p.get_x()+p.get_width()/2., p.get_height()),
        ha='center',
        va='center',
        color='white',
        fontweight='bold',
        xytext=(0, -10),
        textcoords='offset points')


def date_boxplot(df, title, ax = False):
    # Given a dataframe of datestimes, create a boxplot of
    # date distribution
    types = get_var_types()
    # Convert to timestamps
    pdDf = df.select(* (F.unix_timestamp(c).alias(c) for c in types['intervalVars']) ).toPandas()

    if ax:
        pdDf.boxplot(rot=270, figsize=[10,10], ax = ax)
    else:
        ax = pdDf.boxplot(rot=270, figsize=[10,10])

    # Min and Max date plus and minus one month
    max = pd.to_datetime(pdDf.max().max(), unit='s') + pd.DateOffset(months=1)
    min = pd.to_datetime(pdDf.min().min(), unit='s') - pd.DateOffset(months=1)

    # Date labels by month
    yLabels = pd.date_range(start=min.date(), end=max.date(), freq='MS')
    # Convert ticks to unix timestamp (int)
    ytick = [t.value // 10 ** 9 for t in yLabels]
    # Year and month readable labels
    newLabels = [ts.strftime('%Y-%m') for ts in yLabels]
    ax.set_yticks(ytick)
    ax.set_yticklabels(labels=newLabels)

    ax.set_title(title)


def single_val(df):
    """ Give a dataframe, and a column to group by return a list of
        single value variables
    """

    inCols = []

    for c in df.columns:
        if df[c].unique().size == 1:
           inCols.append(c)

    return inCols

def display_sv_cols(df, cols):
    return df.select(cols).toPandas().head(1).transpose()

def display_single_val(df):
    cols = single_val(df.toPandas())
    return display_sv_cols(df, cols)

def id_to_name(df, idVar, newVar, newIdList):
  # Given a dataframe, id variable, new variable name and list of new ids
  # add a new variable to the dataframe mapping the id to the array

  # Save org ids to a list
  oldIdList = [row[idVar] for row in df.select(idVar).distinct().orderBy(idVar).collect()]

    # Create map
  newIdMap = dict()
  # Add letters to map
  for i, val in enumerate(oldIdList):
      newIdMap[val] = newIdList[i]

  # Create mapping expression
  mapping_expr = F.create_map([F.lit(x) for x in chain(*newIdMap.items())])

  # Add org column with letter related to id
  return df.withColumn(newVar, mapping_expr[df[idVar]])


In [1]:
# Return elements in whitelist
def whitelist(l, whitelist):
    if whitelist:
        return l[ np.isIn(whitelist)]
    else:
        return l


# Return a dictionary including arrays of variable names for each category
# If dfColumns provied, return only values in dfColumns
def get_var_cats(dfColumns = False):
    cat = dict()
    dfPd = load_df('dfDesc').toPandas()

    cat['orgVars']               = whitelist( dfPd.loc[ dfPd['category'] == 'Organization' ].field.tolist(), dfColumns)
    cat['sectionVars']           = whitelist( dfPd.loc[ dfPd['category'] == 'Section' ].field.tolist(), dfColumns)
    cat['learnerVars']           = whitelist( dfPd.loc[ dfPd['category'] == 'Learner' ].field.tolist(), dfColumns)
    cat['assessmentVars']        = whitelist( dfPd.loc[ dfPd['category'] == 'Assessment' ].field.tolist(), dfColumns)
    cat['assignmentVars']        = whitelist( dfPd.loc[ dfPd['category'] == 'Assignment' ].field.tolist(), dfColumns)
    cat['itemVars']              = whitelist( dfPd.loc[ dfPd['category'] == 'Item' ].field.tolist(), dfColumns)
    cat['assignmentAttemptVars'] = whitelist( dfPd.loc[ dfPd['category'] == 'Assignment Attempt' ].field.tolist(), dfColumns)
    cat['itemAttemptVars']       = whitelist( dfPd.loc[ dfPd['category'] == 'Item Attempt' ].field.tolist(), dfColumns)

    return cat

# Return a dictionary including arrays of variable names for each type
# If dfColumns provied, return only values in dfColumns
def get_var_types(dfColumns = False):
    type = dict()
    dfPd = load_df('dfDesc').toPandas()

    type['identifierVars']  = whitelist( dfPd.loc[ dfPd['type'] == 'Categorical Identifier' ].field.tolist(), dfColumns)
    type['nominalVars']     = whitelist( dfPd.loc[ dfPd['type'] == 'Categorical Nominal' ].field.tolist(), dfColumns)
    type['continousVars']   = whitelist( dfPd.loc[ dfPd['type'] == 'Numeric Continuous' ].field.tolist(), dfColumns)
    type['intervalVars']    = whitelist( dfPd.loc[ dfPd['type'] == 'Categorical Interval' ].field.tolist(), dfColumns)
    type['binaryVars']      = whitelist( dfPd.loc[ dfPd['type'] == 'Categorical Binary' ].field.tolist(), dfColumns)

    return type


# Given a dataframe of datetime fields, return a matrix of the mean difference
#
def date_diff_map(df, title, scale = 'D', ax = None):

    intVars = df.columns
    intSize = len(intVars)
    am      = pd.DataFrame(np.zeros(shape=(intSize , intSize)), columns = intVars, index = intVars)


    for v1 in intVars:
        for v2 in intVars:
            if v1 != v2:

                mean = ( ( df[v1] - df[v2] ) / np.timedelta64(1,scale) ).mean()
                am.at[v1,v2] = mean

    ax = sn.heatmap(am, annot=True, fmt=".0f", ax=ax)
    ax.set_title(title)


def get_random_sample(df):
    return df.sample(False, .10, 8764664)

def mean_hours_assignment_interval(df, ax=None):
    assignIntVars = [
        'student_start_datetime',
        'was_in_progress_datetime',
        'scored_datetime',
        'was_submitted_datetime_actual',
        'student_stop_datetime',
        'was_fully_scored_datetime',
    ]
    date_diff_map(df.select(assignIntVars).toPandas(), "Mean Hours Between Assignment Interval Vars", 'h', ax)


def dual_mean_hours_assignment(df1, df2, title1='', title2='', main=''):
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, sharex=True)

    mean_hours_assignment_interval(df1, ax1)
    ax1.set_title(title1)

    mean_hours_assignment_interval(df2, ax2)
    ax2.set_title(title2)

    plt.suptitle(main)
    plt.show()

In [ ]:
def impute_749_to_750_null_dates(dfFlt):

    # Get sample to extract means
    pdDf = get_random_sample(dfFlt).select('student_start_datetime', 'student_stop_datetime', 'was_fully_scored_datetime', 'scored_datetime').toPandas()

    # Calculate mean difference in seconds
    mStStartDate = ( (pdDf['scored_datetime'] - pdDf['student_start_datetime'])     / np.timedelta64(1, 's') ).mean()
    mStStopDate  = ( (pdDf['scored_datetime'] - pdDf['student_stop_datetime'])      / np.timedelta64(1, 's') ).mean()
    mWFullScored = ( (pdDf['scored_datetime'] - pdDf['was_fully_scored_datetime'])  / np.timedelta64(1, 's') ).mean()


    return dfFlt.withColumn(
                "student_start_datetime",
                F.when(
                    F.col("student_start_datetime").isNull(),
                    (F.unix_timestamp("scored_datetime") - mStStartDate).cast('timestamp')
                ).otherwise( F.col("student_start_datetime") )
            ).withColumn(
                "student_stop_datetime",
                F.when(
                    F.col("student_stop_datetime").isNull(),
                    (F.unix_timestamp("scored_datetime") - mStStopDate).cast('timestamp')
                ).otherwise( F.col("student_stop_datetime") )
            ).withColumn(
                "was_fully_scored_datetime",
                F.when(
                    F.col("was_fully_scored_datetime").isNull(),
                    (F.unix_timestamp("scored_datetime") - mWFullScored).cast('timestamp')
                ).otherwise( F.col("was_fully_scored_datetime") )
    )


def impute_3422_null_dates(dfFlt):

    # Get sample to extract means
    pdDf = get_random_sample(dfFlt).select('student_start_datetime', 'student_stop_datetime', 'was_fully_scored_datetime', 'scored_datetime').toPandas()

    # Calculate mean difference in seconds
    mScoredDate = ( (pdDf['scored_datetime'] - pdDf['student_stop_datetime'])     / np.timedelta64(1, 's') ).mean()


    return dfFlt.withColumn(
                "scored_datetime",
                F.when(
                    F.col("scored_datetime").isNull(),
                    (F.unix_timestamp("student_stop_datetime") - mScoredDate).cast('timestamp')
                ).otherwise( F.col("scored_datetime") )
    )


def impute_9965_null_dates(df):

    # Get sample to extract means
    pdDf = get_random_sample(df).select('student_start_datetime', 'student_stop_datetime', 'was_in_progress_datetime', 'scored_datetime').toPandas()

    # Calculate mean difference in seconds
    meanDiff = ( (pdDf['was_in_progress_datetime'] - pdDf['student_start_datetime'])     / np.timedelta64(1, 's') ).mean()


    return df.withColumn(
                "scored_datetime",
                F.when(
                    F.col("was_in_progress_datetime").isNull(),
                    (F.unix_timestamp("student_start_datetime") - meanDiff).cast('timestamp')
                ).otherwise( F.col("was_in_progress_datetime") )
    )



# Given a dataframe and variable name, return the value names and counts for that variable
def count_values(df, f):
   return df.groupBy(f).count().orderBy('count', ascending=False)


def response_correctness_bar_plot(df, varName, ax=None):
    pdDf = df.groupBy(varName).count().orderBy('count', ascending=False).toPandas()
    ax = pdDf.plot(kind='bar', ax=ax)
    labels = pdDf[varName].fillna('null').tolist()
    ax.set_xticklabels(labels=labels)
    ax.set_title('Values ' + varName)
    annotate_plot(ax)


# Create a binary catagorical variable for final_score_unweighted
def add_zero_score_var(df):
    return df.withColumn(
                "zero_score",
                  F.when( F.col('final_score_unweighted') == 0, 'Yes').otherwise("No")
                )

# Create a bar chart counting countvar for each groupvar
def num_group_bar_chart(df, groupByVar, countVar, countAlias, title, ax = None):
  sByO = df.groupBy(groupByVar).agg(F.countDistinct(countVar).alias(countAlias)).orderBy(groupByVar)
  pdDf = sByO.toPandas()

  # Add mean
  mean = sByO.agg(F.round(F.avg(F.col(countAlias))).alias('mean')).collect()[0][0]
  # Append row with mean
  pdDf = pdDf.append({groupByVar: 'mean', countAlias: mean}, ignore_index=True)

  axa = pdDf.plot(groupByVar,countAlias, kind='bar', ax=ax, title=title)
  annotate_plot(axa)
  if (ax == None):
    plt.show()


def num_sections_by_org_bar_chart(df, ax = None, title='Num Sections by Organization'):
    num_group_bar_chart(df, 'org_id', 'section_id', 'sections', title, ax)


def num_learners_by_org_bar_chart(df):
    num_group_bar_chart(df, 'org_id', 'learner_id', 'learners', 'Num Learners by Organization')


def mean_group_bar_chart(df, group1, group2, countVar, countAlias, title='', ax=None):
  lByS = df.groupBy(group1, group2).agg(F.countDistinct(countVar).alias(countAlias))
  # Av
  lBySMean = lByS.groupBy(group1).agg( F.avg(countAlias).alias(countAlias) ).orderBy(group1)
  pdDf = lBySMean.toPandas()

  # Add mean
  mean = lBySMean.agg(F.round(F.avg(F.col(countAlias))).alias('mean')).collect()[0][0]
  # Append row with mean
  pdDf = pdDf.append({group1: 'mean', countAlias: mean}, ignore_index=True)

  axo = pdDf.plot.bar(group1,countAlias, ax=ax, title=title)
  annotate_plot(axo)

  if (ax == None):
    plt.show()


def mean_sec_learners_by_org_bar_chart(df):
    mean_group_bar_chart(df, 'org_id','section_id', 'learner_id', 'learners', 'Mean Section Learners by Organization')


def mean_sec_assess_by_org_bar_chart(df, ax=None, title='Mean Section Assessments by Organization'):
    mean_group_bar_chart(df, 'org_id','section_id', 'assessment_id', 'assessments', ax=ax, title=title)


def mean_assess_by_org_bar_chart(df, ax=None, title = 'Mean Learners Assessments by Organization'):
    mean_group_bar_chart(df, 'org_id','learner_id', 'assessment_id', 'assessments',ax=ax, title=title,)

# Mean Scores by Organization
def mean_scores_by_orgs_bar_chart(df):
  sByO = df.groupBy('org_id').agg( F.avg('final_score_unweighted').alias('scores') ).orderBy('org_id')

  pdDf = sByO.toPandas()

  # Add mean
  meanAssess = sByO.agg(F.round(F.avg(F.col('scores'))).alias('mean')).collect()[0][0]
  # Append row with mean
  pdDf = pdDf.append({'org_id': 'mean', 'scores': meanAssess}, ignore_index=True)

  ax = pdDf.plot.bar('org_id','scores', title='Mean Scores by Organization')
  annotate_plot(ax)
  plt.show()


def crosstab_percent(table):
    return table.apply(lambda r: round(r/r.sum() * 100), axis=1)


# Given two dataframes of datetime fields, with optional titles, return two side by side boxplots
#
def dual_date_boxplot(df1, df2, title1='', title2='', main=''):

    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, sharex=True)

    date_boxplot( df1, title1, ax1)
    date_boxplot(df2, title2, ax2)

    plt.suptitle(main)
    plt.show()


# Given two pandas dataframes with one variable each, return two histograms
def dual_hist(pdDf1, pdDf2, title1='', title2='', main=''):

    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)

    pdDf1.hist(ax=ax1)
    ax1.set_title(title1)

    pdDf2.hist(ax=ax2)
    ax2.set_title(title2)

    plt.suptitle(main)
    plt.show()

# Given two dataframes, create two heatmaps of all variables in dataframe
def dual_assoc_heatmap(df1, df2, title1 = '', title2 = '', main = ''):
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True, sharex=True, figsize=(10,5))

    associations( df1.toPandas(), nan_replace_value='null', ax=ax1, plot=False )
    ax1.set_title(title1)

    associations( df2.toPandas(), nan_replace_value='null', ax=ax2, plot=False )
    ax2.set_title(title2)

    plt.suptitle(main)
    plt.show()


# Impute the 4446 null dates with the mean difference of was_submitted_datetime_actual and student_stop_datetime
def impute_4446_null_dates(df):

    # Get sample to extract means
    pdDf = df.select( 'scored_datetime', 'was_submitted_datetime_actual').toPandas()

    # Calculate mean difference in seconds
    meanDiff = ( (pdDf['scored_datetime'] - pdDf['was_submitted_datetime_actual'])     / np.timedelta64(1, 's') ).mean()


    return df.withColumn(
                "was_submitted_datetime_actual",
                F.when(
                    (F.col('was_submitted_datetime_actual').isNull()) & (F.col('final_score_unweighted') > 0),
                    (F.unix_timestamp("scored_datetime") - meanDiff).cast('timestamp')
                ).otherwise( F.col("was_submitted_datetime_actual") )
    )


def impute_number_of_learners(dfClean):

    # Calculate number of learners on Filtered
    dfCount = dfClean.groupBy('assessment_instance_id', 'number_of_learners').agg(
        F.countDistinct('learner_id').alias('number_of_learners_calc')
    ).select('assessment_instance_id', 'number_of_learners_calc')

    # Update with calculated value
    dfClean = dfClean.join(dfCount, on=['assessment_instance_id'], how='left')

    # Drop incorrect number of learners
    dfClean = dfClean.drop('number_of_learners')

    # Rename calculated to original
    return dfClean.withColumnRenamed("number_of_learners_calc","number_of_learners")


# Reduce the number of levels in item_type_code_name
def reduce_type_code_levels(dfClean):
    # Combine fillInTheBlank and FillinBlankResponse
    dfClean = dfClean.withColumn("item_type_code_name", F.when( F.col("item_type_code_name") == "FillinBlankResponse", "fillInTheBlank" ).otherwise(F.col("item_type_code_name")) )

    # Combine multipleChoice and MultipleChoiceResponse
    dfClean = dfClean.withColumn("item_type_code_name", F.when( F.col("item_type_code_name") == "MultipleChoiceResponse", "multipleChoice" ).otherwise(F.col("item_type_code_name")) )

    # Total count
    tot = dfClean.filter(F.col("item_type_code_name").isNull() == False).count()

    freqTable = dfClean.groupBy("item_type_code_name") \
                   .count() \
                   .withColumnRenamed('count', 'cnt_per_group') \
                   .withColumn('perc_of_count_total', ( F.col('cnt_per_group') / tot) * 100 ) \
                   .orderBy("cnt_per_group", ascending=False)

    # freqTable.show(50, False)

    # We only want five levels, so convert everything below 6% to other


    otherRows    = freqTable.filter("perc_of_count_total < 6")
    otherLevels  = [row['item_type_code_name'] for row in otherRows.select("item_type_code_name").collect()]

    return dfClean.withColumn("item_type_code_name", F.when( F.col("item_type_code_name").isin(otherLevels) | F.col("item_type_code_name").isNull() , "Other" ).otherwise(F.col("item_type_code_name")) )